In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50, DenseNet121
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [2]:
#Define as pastas de treino, validação e teste
base_dir = 'chest_xray'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [3]:
datagen = ImageDataGenerator(rescale=1./255) # Normaliza as imagens

train_generator = datagen.flow_from_directory(train_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='binary') # Define o tamanho das imagens e o tamanho do batch de treino

validation_generator = datagen.flow_from_directory(validation_dir,
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='binary') # Define o tamanho das imagens e o tamanho do batch de validação

test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='binary',
                                             shuffle=False) # Define o tamanho das imagens e o tamanho do batch de teste, além de não embaralhar as imagens

# Definindo a função para criar modelos
def create_model(base_model):
    model = models.Sequential() # Cria um modelo sequencial
    model.add(base_model)     # Adiciona a arquitetura pré-treinada
    model.add(layers.Flatten()) # Adiciona uma camada de achatamento
    model.add(layers.Dense(256, activation='relu')) # Adiciona uma camada densa com 256 neurônios
    model.add(layers.Dropout(0.5)) # Adiciona uma camada de dropout com 50% de chance
    model.add(layers.Dense(1, activation='sigmoid')) # Adiciona uma camada densa com 1 neurônio e função de ativação sigmoid

    return model

# Criando modelos com base nas arquiteturas pré-treinadas
vgg16_model = create_model(VGG16(weights='imagenet',
                                 include_top=False,
                                 input_shape=(224, 224, 3))) # Define a arquitetura VGG16 com pesos do ImageNet e sem a camada densa de classificação

inception_model = create_model(InceptionV3(weights='imagenet',
                                           include_top=False,
                                           input_shape=(224, 224, 3))) # Define a arquitetura InceptionV3 com pesos do ImageNet e sem a camada densa de classificação

resnet_model = create_model(ResNet50(weights='imagenet',
                                     include_top=False,
                                     input_shape=(224, 224, 3))) # Define a arquitetura ResNet50 com pesos do ImageNet e sem a camada densa de classificação

densenet_model = create_model(DenseNet121(weights='imagenet',
                                          include_top=False,
                                          input_shape=(224, 224, 3))) # Define a arquitetura DenseNet121 com pesos do ImageNet e sem a camada densa de classificação


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [9]:
import sys
from PIL import Image
sys.modules['Image'] = Image  # Evita um erro de incompatibilidade entre o Pillow e o Keras

In [12]:
#Se der erro, executar os comandos abaixo
conda uninstall PIL
conda uninstall Pillow
conda install Pillow

SyntaxError: invalid syntax (2816729875.py, line 1)

In [4]:
# Função para compilar e treinar o modelo
def compile_and_train(model, train_generator, validation_generator, epochs=10):
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['acc']) # Compila o modelo com a função de perda binary_crossentropy, otimizador Adam e métrica de acurácia
    history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator) # Treina o modelo com os dados de treino e validação
    return model, history

# Treinando os modelos
vgg16_model, vgg16_history = compile_and_train(vgg16_model, train_generator, validation_generator)
inception_model, inception_history = compile_and_train(inception_model, train_generator, validation_generator)
resnet_model, resnet_history = compile_and_train(resnet_model, train_generator, validation_generator)
densenet_model, densenet_history = compile_and_train(densenet_model, train_generator, validation_generator)

Epoch 1/10
163/163 [==============================] - 1085s 7s/step - loss: 0.1928 - acc: 0.9155 - val_loss: 0.0482 - val_acc: 1.0000
Epoch 2/10
163/163 [==============================] - 1068s 7s/step - loss: 0.0743 - acc: 0.9735 - val_loss: 0.0424 - val_acc: 1.0000
Epoch 3/10
163/163 [==============================] - 1063s 7s/step - loss: 0.0644 - acc: 0.9789 - val_loss: 0.1454 - val_acc: 0.9375
Epoch 4/10
163/163 [==============================] - 1059s 6s/step - loss: 0.0542 - acc: 0.9804 - val_loss: 0.0215 - val_acc: 1.0000
Epoch 5/10
163/163 [==============================] - 1059s 6s/step - loss: 0.0302 - acc: 0.9887 - val_loss: 0.2282 - val_acc: 0.8750
Epoch 6/10
163/163 [==============================] - 1058s 6s/step - loss: 0.0372 - acc: 0.9864 - val_loss: 0.0301 - val_acc: 1.0000
Epoch 7/10
163/163 [==============================] - 1056s 6s/step - loss: 0.0197 - acc: 0.9931 - val_loss: 0.0073 - val_acc: 1.0000
Epoch 8/10
163/163 [==============================] - 1102s 7s

In [5]:
# Função para avaliar o modelo
def evaluate_model(model, test_generator):
    test_generator.reset() # Reseta o gerador de teste
    predictions = model.predict(test_generator) # Faz as predições com o gerador de teste
    y_pred = np.round(predictions).astype(int) # Arredonda as predições para 0 ou 1
    y_true = test_generator.classes # Obtém as classes verdadeiras
    report = classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']) # Cria o relatório de classificação
    print(report) # Imprime o relatório

# Avaliando os modelos
print("VGG16:")
evaluate_model(vgg16_model, test_generator)
print("InceptionV3:")
evaluate_model(inception_model, test_generator)
print("ResNet50:")
evaluate_model(resnet_model, test_generator)
print("DenseNet121:")
evaluate_model(densenet_model, test_generator)

VGG16:
20/20 [==============================] - 30s 1s/step
              precision    recall  f1-score   support

      Normal       0.95      0.41      0.57       234
   Pneumonia       0.74      0.99      0.84       390

    accuracy                           0.77       624
   macro avg       0.84      0.70      0.71       624
weighted avg       0.82      0.77      0.74       624

InceptionV3:
20/20 [==============================] - 10s 424ms/step
              precision    recall  f1-score   support

      Normal       0.98      0.23      0.37       234
   Pneumonia       0.68      1.00      0.81       390

    accuracy                           0.71       624
   macro avg       0.83      0.61      0.59       624
weighted avg       0.79      0.71      0.64       624

ResNet50:
20/20 [==============================] - 18s 828ms/step
              precision    recall  f1-score   support

      Normal       1.00      0.04      0.08       234
   Pneumonia       0.64      1.00      0.7